# Word2Vec

In [7]:
from pythainlp.tokenize import word_tokenize

import numpy as np

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import dill as pickle
import pandas as pd

In [1]:
#FOR ANALYSIS/CLEANING/COMPUTATION:
import pandas as pd
import numpy as np
import time
#FOR VISUALIZATION:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
print("Loading data file now, this could take a while depending on file size")
start = time.time()
df = pd.read_csv('prachathai-67k.csv') # ADD-CSV
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds\n")
print("Number of rows : ",df.shape[0]," and the number of columns : ",df.shape[1])
missing_values = df.isna().sum().sum()
duplicated_values = df.duplicated().sum()
print(f'\nMissing values: {missing_values}')
print(f'Duplicated values: {duplicated_values}')
if missing_values >= 1:
    print('\nMissing values by column:')
    print(df.isna().sum())
print("\nUnique Values in Each Column:")
print(df.nunique())

Loading data file now, this could take a while depending on file size
Loading took 10.25 seconds

Number of rows :  67889  and the number of columns :  5

Missing values: 0
Duplicated values: 0

Unique Values in Each Column:
url          67889
date         66732
title        67417
body_text    67753
labels       44365
dtype: int64


In [3]:
df.head()

,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


In [9]:
sentences = [word_tokenize(text, engine='newmm') for text in df["body_text"].tolist()]

In [10]:
sentences[0:1]

[['ประชา',
  'ไท',
  ' ',
  '---',
  ' ',
  '23',
  ' ',
  'ส.ค.',
  '2547',
  '  ',
  'นักวิจัย',
  'ฯ',
  ' ',
  'ชี้',
  'นโยบาย',
  'จีเอ็มโอ',
  'เอื้อ',
  'ต่อ',
  'การค้นคว้าวิจัย',
  ' ',
  'แต่',
  'ไม่',
  'ใช่',
  'คำตอบ',
  'ทั้งหมด',
  ' ',
  'ขณะที่',
  'ผอ.',
  'สวทช.',
  'ระบุ',
  'มาตรการ',
  'ตรวจสอบ',
  'ความปลอดภัย',
  'มี',
  'มา',
  'ก่อน',
  'การ',
  'กำหนด',
  'นโยบาย',
  'ของ',
  'รัฐบาล',
  ' ',
  'เพียงแต่',
  'ทบทวน',
  'และ',
  'ปรับปรุง',
  'เพิ่มเติม',
  '  ',
  '\n',
  'สำนักงาน',
  'พัฒนา',
  'วิทยาศาสตร์',
  'และ',
  'เทคโนโลยี',
  '  ',
  'จัด',
  'สัมมนา',
  'เกี่ยวกับ',
  'มะละกอ',
  'ดัดแปลง',
  'พันธุกรรม',
  'ในประเทศ',
  'ไทย',
  ' ',
  'มี',
  'นักวิชาการ',
  ' ',
  'นักวิจัย',
  'จาก',
  'กรมวิชาการเกษตร',
  '  ',
  'มหาวิทยาลัยเกษตรศาสตร์',
  '  ',
  'และ',
  'มหาวิทยาลัยมหิดล',
  'เข้าร่วม',
  '  ',
  'โดย',
  'มี',
  'การ',
  'นำเสนอ',
  'ผลการศึกษา',
  'ทดลอง',
  'ที่ผ่านมา',
  '  ',
  'ซึ่ง',
  'พบ',
  'ว่า',
  'มะละกอ',
  '\n',
  'จีเอ็ม

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager
# from gensim.models import KeyedVectors

In [14]:
# Join the tokens back into strings
joined_sentences = [' '.join(sentence) for sentence in sentences]

vectorizer = TfidfVectorizer(lowercase=False)
X = vectorizer.fit_transform(joined_sentences)


In [16]:
feature_names = vectorizer.get_feature_names_out()

In [18]:
np.set_printoptions(precision = 2)
for i , tfide_enc in enumerate(X[:2]):
    print(f"Review {i+1} : {tfide_enc}")

Review 1 : <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 208 stored elements and shape (1, 156987)>
  Coords	Values
  (0, 133065)	0.052492841747396116
  (0, 154294)	0.06021008086730769
  (0, 6829)	0.03422894341521772
  (0, 7456)	0.04370185341376014
  (0, 120038)	0.06172367615111905
  (0, 131791)	0.15228768838088572
  (0, 150664)	0.12540062568701507
  (0, 136365)	0.28105205472237876
  (0, 120534)	0.06696869532901412
  (0, 130298)	0.1177111047676531
  (0, 147178)	0.0637967400678189
  (0, 151331)	0.05278229054136575
  (0, 154500)	0.03575588290901923
  (0, 153978)	0.05563918110337353
  (0, 123272)	0.038228185581753255
  (0, 124055)	0.022766276119943786
  (0, 121444)	0.024910322272550692
  (0, 133908)	0.048974961286265814
  (0, 142128)	0.16968985320058028
  (0, 138283)	0.020368758220179063
  (0, 136083)	0.06607078565063851
  (0, 128141)	0.025318184181681396
  (0, 122692)	0.20891459006792226
  (0, 136038)	0.03921785544093228
  (0, 144415)	0.050762876211679325
  :	:
  (0, 1202

In [ ]:
num_workers = 4
downsampling = 1e-3
num_features = 300
min_word_count = 10
context = 8
model_name = "word2vec_gensim"

In [ ]:
w2v_model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)
w2v.build_vocab(sentences)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30)

In [ ]:
WORD = "ประชาธิปไตย"
w2v.wv.most_similar(WORD ,topn=5)